# TODO
- feature engineering sur la datetime (add year, season, weeknumber)
- LSTM w/ Keras
- ARIMA & SARIMA Model

In [1]:
%matplotlib inline
import pandas as pd, numpy as np
import sys, os

import sklearn.preprocessing as pp
import sklearn.decomposition as decomposition
import sklearn.model_selection as ms
import sklearn.metrics as metrics

import sklearn.tree as tree
import sklearn.svm as svm
import sklearn.ensemble as ensemble
import sklearn.neighbors as neighbors
# import xgboost
# import keras
sys.path.append('../')
# import lib.tools
pd.options.display.max_columns = 50

In [2]:
raw_path = '../data/raw/'

# First approach : prediction by removing Nan rows

## Data Preprocessing

In [3]:
def season(df):
    df['month'] = df.index.month
    season_col = df['month']
    season_col = season_col.replace([12, 1, 2], 0)
    season_col = season_col.replace([3, 4, 5], 1)
    season_col = season_col.replace([6, 7, 8], 2)
    season_col = season_col.replace([9, 10, 11], 3)
    ohc = pd.DataFrame(np.eye(4)[season_col], columns=['winter', 'spring', 'summer', 'fall'], index=df.index)
#     for col in ohc:
#         ohc[col] = ohc[col].astype('category')
    return pd.concat([df, ohc], axis=1)

In [4]:
# for file_ in os.listdir(train_path):
file_ = pd.read_csv(raw_path + 'train_1.csv', header=0, delimiter=';',decimal=',',
                    parse_dates=['date'], index_col='date')

file_.dropna(how='any', axis=0, inplace=True)
file_ = file_.rename(columns={'mois':'month'})

#Transform to dummy
dummies = ['ddH10_rose4', 'insee', 'month']
df_dummy = pd.get_dummies(file_, columns=dummies, prefix=dummies)
#     for col in df_dummy:
#         df_dummy[col] = df_dummy[col].astype('category')
file_.drop(['ddH10_rose4'], inplace=True, axis=1)

# Add temporal features
# file_['week'] = file_.index.week # WEEK AS CATEGORY ?
file_ = season(file_) # add season 
file_.drop(['month'], inplace=True, axis=1)

# Add lag operator (shift ce fait par ville)
groupby_cities = file_.groupby('insee')
shift = groupby_cities.tH2_obs.shift(1)
file_['tH2_obs_shift_1'] = shift

# file_.drop(['insee'], inplace=True, axis=1)

In [5]:
file_.describe(include='all')

,insee,tH2_obs,capeinsSOL0,ciwcH20,clwcH20,ffH10,flir1SOL0,fllat1SOL0,flsen1SOL0,flvis1SOL0,hcoulimSOL0,huH2,iwcSOL0,nbSOL0_HMoy,nH20,ntSOL0_HMoy,pMER0,rr1SOL0,rrH20,tH2,tH2_VGrad_2.100,tH2_XGrad,tH2_YGrad,tpwHPA850,ux1H10,vapcSOL0,vx1H10,ech,winter,spring,summer,fall,tH2_obs_shift_1
count,5.327000e+03,5327.000000,5327.000000,5.327000e+03,5327.000000,5327.000000,5327.000000,5.327000e+03,5327.000000,5.327000e+03,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.000000,5327.0,5327.000000,5327.000000,5327.000000,5327.000000,5320.000000
mean,4.390000e+07,10.788417,38.136585,6.145163e-08,0.000005,3.046857,-188174.544653,-3.039357e+04,41036.097838,3.600000e-12,303.012096,82.763922,0.019828,2.160959,0.263442,4.690180,1016.796230,0.082683,0.003299,11.001615,0.977333,0.001718,0.000151,9.715958,1.135103,18.047841,0.576513,1.0,0.270696,0.299606,0.211564,0.218134,10.783609
std,2.246982e+07,5.643908,105.856725,1.265018e-06,0.000034,1.925308,93392.112580,8.048944e+04,74281.351288,4.039347e-28,369.904797,11.906882,0.354949,2.896689,1.260622,3.330917,8.748785,0.498511,0.021998,5.884702,1.930717,0.002731,0.003618,4.654235,4.247374,8.114475,4.429307,0.0,0.444361,0.458128,0.408455,0.413018,5.645779
min,6.088001e+06,-6.100000,0.000000,0.000000e+00,0.000000,0.008230,-503863.736917,-1.453469e+06,-222339.901199,3.600000e-12,0.466309,15.270923,0.000000,0.000000,0.000000,0.000000,982.386670,0.000000,0.000000,-6.967169,-1.794131,-0.011979,-0.017866,-3.398325,-21.681247,1.305881,-14.486823,1.0,0.000000,0.000000,0.000000,0.000000,-6.100000
25%,3.106900e+07,6.800000,0.000000,0.000000e+00,0.000000,1.747876,-254227.141205,-3.704970e+04,-2325.606176,3.600000e-12,47.205749,76.492978,0.000000,0.000000,0.000000,0.685071,1011.634791,0.000000,0.000000,6.725211,-0.499044,0.000136,-0.000899,6.399964,-1.323747,11.708245,-2.545233,1.0,0.000000,0.000000,0.000000,0.000000,6.800000
50%,3.528100e+07,10.500000,1.055184,0.000000e+00,0.000000,2.612889,-206318.953704,-5.037711e+02,21215.950486,3.600000e-12,160.327952,85.248023,0.000000,0.311535,0.000000,5.925654,1017.063334,0.000000,0.000000,10.850965,0.342556,0.001210,0.000362,9.600600,1.057008,16.982351,0.315875,1.0,0.000000,0.000000,0.000000,0.000000,10.500000
75%,6.712400e+07,14.500000,20.415685,0.000000e+00,0.000000,3.835435,-118547.629227,6.857632e+03,71472.542485,3.600000e-12,417.884727,91.687534,0.000000,4.147225,0.000000,7.995833,1022.381866,0.000000,0.000000,14.962453,2.027604,0.003065,0.001908,13.065874,3.182763,23.206563,3.265235,1.0,1.000000,1.000000,0.000000,0.000000,14.500000
max,7.511400e+07,29.799999,1083.809402,4.305547e-05,0.000413,15.310204,78624.299681,9.387913e+04,641199.521351,3.600000e-12,3123.016425,99.997908,10.107500,8.000000,8.331533,8.000000,1043.259009,14.344218,0.727914,30.683831,11.169983,0.019835,0.014316,21.767814,23.799293,53.544544,22.807208,1.0,1.000000,1.000000,1.000000,1.000000,29.799999


In [6]:
file_.head(14)

,insee,tH2_obs,capeinsSOL0,ciwcH20,clwcH20,ffH10,flir1SOL0,fllat1SOL0,flsen1SOL0,flvis1SOL0,hcoulimSOL0,huH2,iwcSOL0,nbSOL0_HMoy,nH20,ntSOL0_HMoy,pMER0,rr1SOL0,rrH20,tH2,tH2_VGrad_2.100,tH2_XGrad,tH2_YGrad,tpwHPA850,ux1H10,vapcSOL0,vx1H10,ech,winter,spring,summer,fall,tH2_obs_shift_1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-01-01,6088001,5.5,0.198757,0.0,0.000000e+00,1.970260,-386960.595711,-94260.682523,-40545.510054,3.600000e-12,66.600368,79.211738,0.0,0.000000,0.00000,4.173220,1019.543734,0.000000,0.000000,9.034338,-0.084999,0.007725,-0.013024,3.852825,1.330320,10.664653,-2.853134,1,1.0,0.0,0.0,0.0,NaN
2014-01-01,31069001,4.8,8.590036,0.0,0.000000e+00,2.755962,-200181.123871,2282.951687,14399.918192,3.600000e-12,57.954614,83.862960,0.0,0.012709,0.00000,5.736559,1018.290035,0.000000,0.000000,3.853128,1.396138,0.005036,0.000810,8.476699,-2.040020,15.462043,3.019614,1,1.0,0.0,0.0,0.0,NaN
2014-01-01,33281001,8.9,69.203856,0.0,1.026762e-06,3.411283,-62310.340297,9483.691316,64790.365136,3.600000e-12,202.985582,95.711708,0.0,4.792424,0.15494,7.844538,1016.139756,0.081315,0.005008,8.585184,0.387329,0.001372,0.000010,7.643070,3.029610,17.343600,4.889849,1,1.0,0.0,0.0,0.0,NaN
2014-01-01,35281001,6.8,65.270316,0.0,0.000000e+00,4.655633,-61612.120075,-48356.475831,127450.659115,3.600000e-12,343.974193,90.178385,0.0,7.074669,0.00000,7.377704,1009.321502,0.029946,0.008139,7.304130,-0.089267,0.000401,-0.000067,7.089030,-1.027446,13.981847,8.079973,1,1.0,0.0,0.0,0.0,NaN
2014-01-01,59343001,5.4,5.932505,0.0,0.000000e+00,5.855930,-39553.109955,-1022.914242,117800.171480,3.600000e-12,656.972741,92.680476,0.0,7.995791,0.00000,7.995791,1008.724301,0.000000,0.000000,5.829715,-0.489606,0.000204,-0.000191,4.546075,2.802554,9.941609,7.955489,1,1.0,0.0,0.0,0.0,NaN
2014-01-01,67124001,2.9,0.000000,0.0,0.000000e+00,6.697897,-232184.640842,-48990.059882,190374.113081,3.600000e-12,128.078417,79.197440,0.0,0.000000,0.00000,2.729430,1015.678380,0.000000,0.000000,1.694268,3.221535,0.000155,0.000915,7.415663,2.217511,11.333324,9.243589,1,1.0,0.0,0.0,0.0,NaN
2014-01-01,75114001,7.0,13.271466,0.0,2.494773e-07,3.185958,-25228.272328,-18513.392636,-3158.361810,3.600000e-12,548.721209,91.507594,0.0,7.978226,0.02522,7.978226,1011.678651,0.000025,0.000000,7.208069,-1.081230,0.000871,0.001567,4.845582,2.340020,10.430807,7.030941,1,1.0,0.0,0.0,0.0,NaN
2014-01-02,6088001,7.9,2.588260,0.0,0.000000e+00,2.608449,-154865.268269,-123893.703248,-55913.162861,3.600000e-12,149.373482,84.961316,0.0,5.483689,0.00000,7.209474,1016.285475,0.000149,0.000011,10.600003,-0.337017,0.001096,-0.004172,8.109257,0.809966,17.077868,-4.782408,1,1.0,0.0,0.0,0.0,5.5
2014-01-02,31069001,9.9,0.777403,0.0,0.000000e+00,4.282281,-140018.547252,-20709.302019,129016.229110,3.600000e-12,2017.266169,86.036733,0.0,5.099234,0.00000,8.000000,1008.707113,0.046678,0.023633,9.768346,0.051136,0.001342,0.000109,10.796253,-7.049838,21.103269,6.360897,1,1.0,0.0,0.0,0.0,4.8


## Factor Analysis

In [97]:
# file_['2014-01'][]
# calculer la variation moyenne de temperature en de décembre à janvier pour estimer 2014-01-01

,insee,tH2_obs,capeinsSOL0,ciwcH20,clwcH20,ffH10,flir1SOL0,fllat1SOL0,flsen1SOL0,flvis1SOL0,hcoulimSOL0,huH2,iwcSOL0,nbSOL0_HMoy,nH20,ntSOL0_HMoy,pMER0,rr1SOL0,rrH20,tH2,tH2_VGrad_2.100,tH2_XGrad,tH2_YGrad,tpwHPA850,ux1H10,vapcSOL0,vx1H10,ech,winter,spring,summer,fall,tH2_obs_shift_1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-12-31,6088001,3.7,0.244917,0.000000,0.000000e+00,2.605428,-373489.263692,-247814.929511,-96969.479670,3.600000e-12,179.896358,58.991284,0.000000,0.030788,0.000000,7.098075,1025.464668,0.000000,0.0,8.075482,0.429342,0.004432,-0.013341,3.410308,0.784919,9.203935,-5.233326,1,1.0,0.0,0.0,0.0,3.9
2014-12-31,31069001,-1.6,0.000000,0.000000,1.059703e-08,1.483372,-242506.761378,3762.424342,2080.547043,3.600000e-12,46.195980,93.319007,0.858902,0.318870,0.000174,0.318870,1037.323665,0.000000,0.0,-1.834280,2.037842,-0.001091,0.001028,2.739987,1.939617,5.557360,-0.506246,1,1.0,0.0,0.0,0.0,0.0
2014-12-31,33281001,-0.6,0.000000,0.000000,0.000000e+00,1.465156,-249226.767159,21020.394264,50984.894984,3.600000e-12,13.144462,92.462152,0.000000,0.448008,0.000000,0.448008,1037.774480,0.000000,0.0,-1.397170,1.536541,0.004640,-0.001823,2.741799,-4.113913,4.743057,-1.855670,1,1.0,0.0,0.0,0.0,-2.9
2014-12-31,35281001,-3.9,0.384621,0.000042,1.927945e-04,0.831035,-68119.632300,-11593.264986,-40021.282273,3.600000e-12,24.531844,95.714873,0.125000,5.940963,8.011596,5.941333,1038.340179,0.007219,0.0,-3.100147,4.043179,-0.000760,0.002097,1.644502,-1.822777,5.114907,-0.483948,1,1.0,0.0,0.0,0.0,-1.9
2014-12-31,59343001,4.2,3.063417,0.000000,0.000000e+00,1.824854,-258609.312362,7552.826714,52905.214951,3.600000e-12,72.155089,86.523353,0.000000,1.260752,0.000000,1.260752,1035.686000,0.000000,0.0,2.532746,3.531705,0.000869,0.001612,0.882939,4.617304,8.171677,0.080017,1,1.0,0.0,0.0,0.0,3.0
2014-12-31,67124001,-2.0,0.017694,0.000032,1.883901e-04,1.523592,-131545.596140,11198.040838,21751.185823,3.600000e-12,21.078769,99.681236,10.107500,6.711318,8.012889,6.711318,1037.248021,0.003761,0.0,-3.938359,2.204154,0.002834,-0.001422,-1.303072,-1.004775,7.361689,2.604990,1,1.0,0.0,0.0,0.0,-2.8
2014-12-31,75114001,5.4,0.432291,0.000000,0.000000e+00,1.462988,-62084.761886,4646.327473,56103.405488,3.600000e-12,1141.580445,69.720779,0.000000,2.683587,0.000000,2.687695,1037.556800,0.000000,0.0,4.364115,0.963472,0.001485,0.000880,0.801018,3.447337,5.735766,-3.239660,1,1.0,0.0,0.0,0.0,4.1


In [98]:
file_.dropna(how='any', axis=0, inplace=True)
fa = decomposition.FactorAnalysis().fit(file_.drop('tH2_obs'))

In [99]:
fa

FactorAnalysis(copy=True, iterated_power=3, max_iter=1000, n_components=None,
        noise_variance_init=None, random_state=0, svd_method='randomized',
        tol=0.01)

## ML

In [ ]:
seed = 1
results = []
names = []
predictions = [] #table of prediction for each model

In [ ]:
models = []
models.append(('CART', tree.DecisionTreeRegressor(n_jobs=3)))
models.append(('RF', ensemble.RandomForestRegressor(n_jobs=3)))
models.append(('KNN', neighbors.KNeighborsRegressor(n_jobs=3)))
models.append(('SVM', svm.SVR(n_jobs=3)))
models.append(('GB', ensemble.GradientBoostingRegressor(n_jobs=3)))
# models.append(('NB'), ) # Naive Bayes for modeling uncertainty
# LSTM

Apply forward chaining

In [14]:
first_train = True
for i in range(1,36):
    train_file = train_path + 'train_' + str(i) + '.csv'
    test_file = train_path + 'train_' + str(i+1) + '.csv'
    
    x_test = pd.read_csv(test_file, header=0, delimiter=';',decimal=',',
                    parse_dates=['date'], index_col='date').drop('tH2_obs', axis=1)
    y_test = pd.read_csv(test_file, header=0, delimiter=';',decimal=',',
                    parse_dates=['date'], index_col='date', usecols=['tH2_obs'])
    train = pd.read_csv(train_file, header=0, delimiter=';',decimal=',',
                    parse_dates=['date'], index_col='date')
    
    
    if first_train is True:
        x_train = train.drop('tH2_obs', axis=1)
        y_train = train.pop('tH2_obs')
        first_train = False
    else:
        x_train = pd.concat([x_train, train.drop('tH2_obs', axis=1)])
        y_train = pd.concat([y_train, train.pop('tH2_obs')])
        
    for name, m in models: 
    pred = ms.cross_val_predict(m, dff[features], dff[label], cv=kfold, n_jobs=3)
    cv_res = ms.cross_val_score(m, dff[features], dff[label], cv=kfold, n_jobs=3, scoring = 'accuracy')
    m.fit
    
    predictions.append(pred)
    results.append(cv_res)
    names.append(name)
    print "Score %s: %.3f%%, %.3f%%" % (name, cv_res.mean()*100, cv_res.std()*100)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


## Feature engineering

### TODO
- Travail d'une force : vitesse * direction  
https://fr.wikipedia.org/wiki/Travail_d%27une_force


# Draft

In [29]:
groupby_cities = file_.groupby('insee')

Moving Average

In [30]:
pd.DataFrame(groupby_cities.tH2_obs.rolling(window=2).mean())

tH2_obs
insee    date                 
31069001 2014-01-01        NaN
         2014-01-02   7.350000
         2014-01-03  10.700000
         2014-01-04   9.900000
         2014-01-05   6.800000
         2014-01-06   6.950000
         2014-01-07   9.750000
         2014-01-08  11.150000
         2014-01-09  11.450000
         2014-01-10   9.550000
         2014-01-11   8.650000
         2014-01-12   9.900000
         2014-01-13   9.750000
         2014-01-14   6.000000
         2014-01-15   3.950000
         2014-01-16   6.350000
         2014-01-17   5.500000
         2014-01-18   3.450000
         2014-01-19   3.350000
         2014-01-20   4.700000
         2014-01-21   5.600000
         2014-01-22   3.300000
         2014-01-23   4.050000
         2014-01-24   7.650000
         2014-01-25   9.700000
         2014-01-26  10.550000
         2014-01-27  10.150000
         2014-01-28   6.700000
         2014-01-29   4.650000
         2014-01-30   5.100000
...                        ...
75114001 2016-05-01   7.050000
         2016-05-02   7.400000
         2016-05-03   9.950000
         2016-05-04  10.000000
         2016-05-05  10.150000
         2016-05-06  12.350000
         2016-05-07  14.300000
         2016-05-08  15.700000
         2016-05-09  16.900000
         2016-05-10  16.550000
         2016-05-11  15.050000
         2016-05-12  14.600000
         2016-05-13  14.450000
         2016-05-14  12.500000
         2016-05-15   9.050000
         2016-05-16   9.550000
         2016-05-17  10.750000
         2016-05-18  12.250000
         2016-05-19  11.900000
         2016-05-20  10.900000
         2016-05-21  13.650000
         2016-05-22  15.300000
         2016-05-23  13.450000
         2016-05-24  10.900000
         2016-05-25  10.050000
         2016-05-26  11.400000
         2016-05-27  15.000000
         2016-05-28  16.750000
         2016-05-29  16.650001
         2016-05-30  15.850000

[6167 rows x 1 columns]

Interpolate NA val

In [ ]:
groupby_city.capeinsSOL0.apply(pd.Series.interpolate)

# Documentation

- Handling missing val for RF  
https://stats.stackexchange.com/questions/98953/why-doesnt-random-forest-handle-missing-values-in-predictors/186264#186264  https://github.com/scikit-learn/scikit-learn/issues/5870  